In [1]:
import gurobipy

In [2]:
# 前置信息

# 被违反的，需要修复的Policy
POLICY = gurobipy.tuplelist([('m', 'j'), ])

CONFIG = {
    'a': ['i'],
    'b': ['i'],
    'c': [],
    'd': ['f'],
    'e': ['g'],
    'f': ['j'],
    'g': ['j'],
    'h': ['j'],
    'i': ['c', 'd', 'e'],
    'j': ['k'],
    'k': [],
    'm': ['p'],
    'p': [],
}

TOPO = {
    'a': ['i'],
    'b': ['i'],
    'c': ['i'],
    'd': ['i', 'f'],
    'e': ['i', 'g'],
    'f': ['d', 'j'],
    'g': ['e', 'j'],
    'h': ['j'],
    'i': ['a', 'b', 'c', 'd', 'e'],
    'j': ['f', 'g', 'h', 'k', 'q'],
    'k': ['j'],
    'm': ['p'],
    'p': ['m', 'q', 'r'],
    'q': ['j', 'p', 'r'],
    'r': ['q', 'p']
}


# 边集
def get_edge_set(graph):
    result = gurobipy.tuplelist()
    for pre, nexts in graph.items():
        for next in nexts:
            result.append((pre, next))
    return result


E_topo = get_edge_set(TOPO)
E_config = get_edge_set(CONFIG)

由于后面要使用**图的边数**、**结点的邻居**。因此**使用邻接表建有向图**，并**根据图获得边集**

In [3]:
# 创建模型与变量
MODEL = gurobipy.Model()
x_ij = MODEL.addVars(E_topo, vtype=gurobipy.GRB.BINARY, name='x_ij')
x_ijpq = MODEL.addVars(E_topo, POLICY, vtype=gurobipy.GRB.BINARY)
# 更新变量环境
MODEL.update()

In [4]:
# 目标函数
MODEL.setObjective(gurobipy.quicksum(x_ij[i, j] for (i, j) in E_topo if (i, j) not in E_config),
                   sense=gurobipy.GRB.MINIMIZE)

In [5]:
# 创建约束
# (1) & (2) 上面存的TOPO是双向边，所以不用显式声明(2)的Constrain
MODEL.addConstrs(
    gurobipy.quicksum(x_ijpq[i, j, p, q] / len(POLICY) for p, q in POLICY)
    <= x_ij[i, j] for i, j in E_topo
)
MODEL.addConstrs(
    gurobipy.quicksum(x_ijpq[i, j, p, q] for p, q in POLICY)
    >= x_ij[i, j] for i, j in E_topo
)

# (3)
MODEL.addConstrs(
    x_ij[i, j] + x_ij[j, i] <= 1
    for i, j in E_topo
)

# (4)
MODEL.addConstrs(
    gurobipy.quicksum(x_ij[i,j] for j in TOPO[i]) <=1
    for i in TOPO
)

# (5) (6) i=p
MODEL.addConstrs(
    gurobipy.quicksum(x_ijpq[i,j,p,q] for j in TOPO[i]) == 1
    for p,q in POLICY for i in TOPO if i==p
)

# (7) (8) i=q
MODEL.addConstrs(
    gurobipy.quicksum(x_ijpq[j,i,p,q] for j in TOPO[i]) == 1
    for p,q in POLICY for i in TOPO if i==q
)

# (9)
MODEL.addConstrs(
    gurobipy.quicksum((x_ijpq[i,j,p,q]-x_ijpq[j,i,p,q]) for j in TOPO[i]) == 0
    for p,q in POLICY for i in TOPO if i != p and i != q
)

MODEL.update()

In [6]:
# 执行
MODEL.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 126 rows, 64 columns and 282 nonzeros
Model fingerprint: 0x6731fe99
Variable types: 0 continuous, 64 integer (64 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 126 rows and 64 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 1: 2 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.000000000000e+00, best bound 2.000000000000e+00, gap 0.0000%


In [10]:
# 输出
if MODEL.status == gurobipy.GRB.Status.OPTIMAL:
    for k, v in MODEL.getAttr('x', x_ij).items():
        if v == 1 and (k not in E_config):
            print(f'添加边: {k}')

添加边: ('p', 'q')
添加边: ('q', 'j')
